In [1]:
from graph_tool.all import *
from tqdm import tqdm
import csv

In [2]:
# get dict with REMID as key and corresponding CREM as value
# dict with CREM as key and contained REMs as value
crem_dict = dict()
rem_crem_dict = dict()
with open('../data/clusterREMs_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        if row[0] != "REMID":
            crem_dict[row[0]] = [row[1], row[3], row[4]]
            if rem_crem_dict.get(row[1]) == None:
                rem_crem_dict[row[1]] = [row[0]]
            else:
                rem_crem_dict[row[1]].append(row[0])

1264526it [00:05, 247458.85it/s]


In [3]:
# get dict with GENE name as key and start and end as value
gene_dict = dict()
with open('../data/GeneAnnotation_1.csv') as file:
    reader = csv.reader(file)
#     counter = 0
    for row in tqdm(reader):
        if row[0] != "chr":
            gene_dict[row[3]] = [int(row[1]), int(row[2]), row[0]]
#             print(row)
#             counter += 1
#             if(counter == 10):
#                 break

58175it [00:00, 467508.88it/s]


58174

In [11]:
g = Graph()
name = g.new_vp("string")
g.vp.name = name
type = g.new_vp("string")
g.vp.type = type
rem = g.new_ep("string")
g.ep.rem = rem
chr = g.new_vp("string")
g.vp.chr = chr
start = g.new_vp("int")
g.vp.start = start
end = g.new_vp("int")
g.vp.end = end
score = g.new_ep("double")
g.edge_properties["score"] = score
p_value = g.new_ep("double")
g.ep.p_value = p_value

done_proteins_dict = {}
done_crems = {}
with open('../data/REMAnnotationModelScore_1.csv') as file:
    reader = csv.reader(file)
#     counter = 0
    for row in tqdm(reader):
        rem_name = row[4]
        gen = row[3]
        crem = crem_dict.get(rem_name)
        if crem != None:
            crem = crem[0]
        if row[1] != "start":
            if crem in done_crems:
                v1 = g.vertex(done_crems.get(crem))
            elif crem != None:
                v1 = g.add_vertex()
                g.vp.name[v1] = crem
                g.vp.type[v1] = "REM"
                g.vp.chr[v1] = row[0]
                g.vp.start[v1] = crem_dict.get(rem_name)[1]
                g.vp.end[v1] = crem_dict.get(rem_name)[2]
                done_crems[crem] = int(v1)
            else:
                v1 = g.add_vertex()
                g.vp.name[v1] = rem_name
                g.vp.type[v1] = "REM"
                g.vp.chr[v1] = row[0]
                g.vp.start[v1] = int(row[1])
                g.vp.end[v1] = int(row[2])
            
            if done_proteins_dict.get(gen) == None:
                v2 = g.add_vertex()
                g.vp.name[v2] = gen
                g.vp.type[v2] = "Gene"
                g.vp.chr[v2] = gene_dict.get(gen)[2]
                g.vp.start[v2] = gene_dict.get(gen)[0]
                g.vp.end[v2] = gene_dict.get(gen)[1]
                done_proteins_dict[gen] = int(v2)
            else:
                v2 = g.vertex(done_proteins_dict.get(gen))
                
            e = g.add_edge(v1, v2)
            g.ep.score[e] = row[7]
            g.ep.p_value[e] = row[6]
            g.ep.rem[e] = rem_name
        
#         counter += 1
#         if(counter == 4):
#             break

2404862it [03:30, 11438.34it/s]


In [12]:
g.save("../data/graph-tool/allCelltypeGraph.gt")